<a href="https://colab.research.google.com/github/sidhunt/CUDA-sample/blob/main/CUDA_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sun Nov 15 16:48:28 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-t0h48fux
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-t0h48fux
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=0c12fb21090ee0a9c6bc16f026aa2b9b419bdca5a4a6bcb9b87b1297ec96a14b
  Stored in directory: /tmp/pip-ephem-wheel-cache-ud59migv/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
int main(void) {
    void MatrixMultiplication(float *, float *, float *, int);
    const int Width = 5;
    float M[Width*Width], N[Width*Width], P[Width*Width];
    for(int i = 0; i < (Width*Width) ; i++) {
        M[i] = rand();
        N[i] = rand();
        P[i] = 0;
    }
    MatrixMultiplication(M, N, P, Width);
    for(int i = 0; i < (Width*Width) ; i++) {
        printf("%f \n", P[i]);
    }
    int quit;
    scanf("%f",&quit);
    return 0;
}

//Matrix multiplication kernel - thread specification
__global__ void MatrixMulKernel(float *Md, float *Nd, float *Pd, int Width) {
    //2D Thread ID
    int tx = threadIdx.x;
    int ty = threadIdx.y;

    //Pvalue stores the Pd element that is computed by the thread
    float Pvalue = 0;

    for(int k = 0; k < Width ; ++k) {
        float Mdelement = Md[ty*Width + k];
        float Ndelement = Nd[k*Width + tx];
        Pvalue += (Mdelement*Ndelement);
    }

    Pd[ty*Width + tx] = Pvalue;
}

void MatrixMultiplication(float *M, float *N, float *P, int Width) {
    int size = Width*Width*sizeof(float);
    float *Md, *Nd, *Pd;

    //Transfer M and N to device memory
    cudaMalloc((void**)&Md, size);
    cudaMemcpy(Md,M,size,cudaMemcpyHostToDevice);
    cudaMalloc((void**)&Nd, size);
    cudaMemcpy(Nd,N,size,cudaMemcpyHostToDevice);

    //Allocate P on the device
    cudaMalloc((void**)&Pd,size);

    //Setup the execution configuration
    dim3 dimBlock(Width,Width);
    dim3 dimGrid(1,1);

    //Launch the device computation threads!
    MatrixMulKernel<<<dimGrid,dimBlock>>>(Md,Nd,Pd,Width);

    //Transfer P from device to host
    cudaMemcpy(P,Pd,size,cudaMemcpyDeviceToHost);

    //Free device matrices
    cudaFree(Md);
    cudaFree(Nd);
    cudaFree(Pd);
}

7103856116374700032.000000 
9928451460352180224.000000 
7021885875256557568.000000 
8705690076660629504.000000 
7664643880179990528.000000 
5955617981546364928.000000 
7603048139280351232.000000 
5504138715972239360.000000 
6827883645605249024.000000 
6080487318089629696.000000 
4652173884852797440.000000 
5250220799188533248.000000 
2821878725623152640.000000 
5406286029390872576.000000 
5418465869447561216.000000 
6372429645496713216.000000 
7580153558410985472.000000 
5903558854750240768.000000 
7617775547778596864.000000 
6703435422015422464.000000 
5809441758924242944.000000 
5327364184260739072.000000 
4761130539363074048.000000 
7251471599717580800.000000 
5929833884119203840.000000 



In [ ]:
%%cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include<time.h>
#include <stdio.h>
#include <sys/time.h>
void MM(float *,float *,float *,int);
 const int m=500;
 const   int n=500;
 const int j=500;
 const   int k=400;
 const   int N=50;
 struct timeval t1,t2;
    float* A=new float[m*n];
  float* B=new float[j*k];
  float* C=new float[m*k];
  float* result=new float[m*k];
void initA(){
    int i=0;
        for( i=0;i<(m*n);i++){
        A[i]=rand()%100;    
    }
}


__global__ void MMKernel(float *Ad,float *Bd,float *Cd, int num){
    int tx=threadIdx.x;
    int ty= threadIdx.y;
    float Cval=0;
    for(int z=0;z<num;z++){
        float Adele=Ad[ty*num+z];
        float Bdele=Bd[z*num+tx];
        Cval +=(Adele*Bdele);
    }
    Cd[ty*num+tx]=Cval;
}
void MM(float *A,float *B,float *C,int num,int count){
    float *Ad, *Bd, *Cd;
    cudaMalloc(&Ad,m*n);
    cudaMemcpy(Ad,A,m*n,cudaMemcpyHostToDevice);
    cudaMalloc(&Bd,j*k);
    cudaMemcpy(Bd,B,j*k,cudaMemcpyHostToDevice);
    cudaMalloc(&Cd,m*k);
    dim3 blocksPerGrid(100,10,2);
    dim3 threadsPerBlock(512,2,1);
    gettimeofday(&t1,0);
    MMKernel<<<blocksPerGrid,threadsPerBlock>>>(Ad,Bd,Cd,num);
    gettimeofday(&t2,0);
   // double executed=(1000000.0*(t2.tv_sec-t1.tv_sec)+t2.tv_usec-t1.tv_usec)/1000.0;
  //  printf("GPU Execution time: %3.4f ms \n",executed);
    cudaMemcpy(C,Cd,m*k,cudaMemcpyDeviceToHost);
    //printf("A%d*B=%0.2f \n",(count+1),C[count]);
    cudaFree(Ad);
    cudaFree(Bd);
    cudaFree(Cd);
}
void execCPU(){
        gettimeofday(&t1,0);
    int sum=0;
     for (int i1 = 0; i1 < m; ++i1) {
      for (int j1 = 0; j1 < k; ++j1) {
         result [i1*N+j1] = 0;
      }
   }

   
   for (int i1 = 0; i1 < m; ++i1) {
      for (int j1 = 0; j1 < j; ++j1) {
         for (int k1 = 0; k1 < n; ++k1) {
           sum  += A[i1*N+k1] * B[k1*N+j1];
         }
         result[i1*N+j1]=sum;
         sum=0;
      }
   }
       gettimeofday(&t2,0);

  //  double executed1=(1000000.0*(t2.tv_sec-t1.tv_sec)+t2.tv_usec-t1.tv_usec)/1000.0;
  //  printf("CPU Execution time: %3.4f ms \n",executed1);
}
int main(void){
    int x=0;
    int a=0;
    int count=0;
            srand(time(NULL));
    for( x=0;x<(j*k);x++){
        B[x]=rand()%100;
        
    }
    for(a=0;a<N;a++,count++){
            initA();
        execCPU();
        MM(A,B,C,N,count);
        
    }
    int abc;
    for(abc=0;abc<m*k;++abc){
        if (result[abc]==C[abc]){
            printf("True..is matching");
        }
            else{
            printf("False");    
            }
        }
    
    return 0;
}

KeyboardInterrupt: ignored

In [ ]:
%%cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include<time.h>
#include <stdio.h>
#include <sys/time.h>
using namespace std;
void MM(float *,float *,float *,int,int,int,int,int,int);
 const int hA=500; //hA
 const   int wA=500;//wA
 const int hB=500;//hB
 const   int wB=400;//wB
 const   int N=100;
 struct timeval t1,t2;
float* A=new float[hA*wA];
float* B=new float[hB*wB];
  float* C=new float[hA*wB];
  float* result=new float[hA*wB];
void initA(int m2, int n2){
    // gettimeofday(&t1,0);
    //            srand(((t1.tv_sec * 1000) + (t1.tv_usec / 1000)));
    int i2=0;
        for(i2=0 ;i2<(m2*n2);i2++){      
        A[i2]=(rand()%10);    
    }
}
void initResC(int m1 ,int k1){
    int c1=0;
    for(c1=0;c1<(m1*k1);c1++){
        result[c1]=0.0;
        C[c1]=0.0;
    }
}

__global__ void MMKernel(float *Ad,float *Bd,float *Cd,const int wA1,const int wB1){
    float Cval=0.0;
    int r=blockIdx.y*blockDim.y+threadIdx.y;
    int c=blockIdx.x*blockDim.x+threadIdx.x;
    for(int z=0;z<wA1;z++){
        float Adele=Ad[r*wA1+z];
        float Bdele=Bd[z*wB1+c];
        Cval +=(float)(Adele*Bdele);
    }
    Cd[r*wB1+c]=Cval;
}
void MM(float *A,float *B,float *C,int num,int count,const int m,const int n,const int j,const int k){
    float *Ad=new float[m*n];
    float *Bd=new float[j*k];
    float *Cd=new float[m*k];
    cudaMalloc(&Ad,sizeof(int)*m*n);
    cudaMemcpy(Ad,A,sizeof(int)*m*n,cudaMemcpyHostToDevice);
    cudaMalloc(&Bd,sizeof(int)*j*k);
    cudaMemcpy(Bd,B,sizeof(int)*j*k,cudaMemcpyHostToDevice);
    cudaMalloc(&Cd,sizeof(int)*m*k);
    cudaMemcpy(Cd,C,sizeof(int)*m*k,cudaMemcpyHostToDevice);
   // dim3 blocksPerGrid(1,1,1);
   dim3 dimBlock(2,1,1);
   dim3 dimGrid;
    dimGrid.x = ((k + dimBlock.x - 1)/dimBlock.x);
    dimGrid.y = ((m + dimBlock.y - 1)/dimBlock.y);
    
    gettimeofday(&t1,0);
    MMKernel<<<dimGrid,dimBlock>>>(Ad,Bd,Cd,n,k);
    gettimeofday(&t2,0);
   // double executed=(1000000.0*(t2.tv_sec-t1.tv_sec)+t2.tv_usec-t1.tv_usec)/1000.0;
  //  printf("GPU Execution time: %3.4f ms \n",executed);
    cudaMemcpy(C,Cd,sizeof(int)*m*k,cudaMemcpyDeviceToHost);
    //printf("A%d*B=%0.2f \n",(count+1),C[count]);
    cudaFree(Ad);
    cudaFree(Bd);
    cudaFree(Cd);
}
void execCPU(float* result,float* A,float* B,int hA2,int wA2,int wB2){
        gettimeofday(&t1,0);
        int i1,j1,k1;
        double a1,b1;
 for( i1=0;i1<hA2;i1++){
     for(j1=0;j1< wB2;j1++){
         int sum=0;
         for( k1=0;k1<wA2;k1++){
             a1 =A[i1*wA2+k1];
             b1=B[k1*wB2+j1];
             sum+= a1*b1;
         }
         result[i1*wB2+j1]=sum;
     }
 }
       gettimeofday(&t2,0);

  //  double executed1=(1000000.0*(t2.tv_sec-t1.tv_sec)+t2.tv_usec-t1.tv_usec)/1000.0;
  //  printf("CPU Execution time: %3.4f ms \n",executed1);
}
int main(void){
    int x1=0;
    int a1=0;
    int count1=0;
                        srand(time(NULL));
    for( x1=0;x1<(hB*wB);x1++){
        B[x1]=(rand()%10);
        
    }
    for(a1=0;a1<N;a1++,count1++){
            initA(hA,wA);
            initResC(hA,wB);
        execCPU(result,A,B,hA,wA,wB);
       MM(A,B,C,N,count1,hA,wA,hB,wB);
        
    }
    //int abc;
    //for(abc=0;abc<hA*wB;abc++){
       // if (result[abc]==C[abc]){
    //        printf("%0.1f===%0.1f\n",result[abc],C[abc]);
      //  }
    //    }
    
    return 0;
}

Streaming output truncated to the last 5000 lines.
10040.0===10040.0
10199.0===10199.0
10592.0===10592.0
9649.0===9649.0
9779.0===9779.0
9754.0===9754.0
9603.0===9603.0
9350.0===9350.0
9369.0===9369.0
10110.0===10110.0
10117.0===10117.0
10402.0===10402.0
9813.0===9813.0
10641.0===10641.0
9945.0===9945.0
9382.0===9382.0
9915.0===9915.0
9543.0===9543.0
9467.0===9467.0
9897.0===9897.0
9615.0===9615.0
9411.0===9411.0
9650.0===9650.0
9993.0===9993.0
9779.0===9779.0
8969.0===8969.0
9749.0===9749.0
9902.0===9902.0
9334.0===9334.0
9998.0===9998.0
10247.0===10247.0
9673.0===9673.0
9972.0===9972.0
9886.0===9886.0
10123.0===10123.0
10476.0===10476.0
8995.0===8995.0
9636.0===9636.0
9750.0===9750.0
9798.0===9798.0
9525.0===9525.0
8813.0===8813.0
9968.0===9968.0
10166.0===10166.0
10003.0===10003.0
9576.0===9576.0
9814.0===9814.0
9540.0===9540.0
10337.0===10337.0
9797.0===9797.0
10323.0===10323.0
9707.0===9707.0
9701.0===9701.0
10072.0===10072.0
9623.0===9623.0
8684.0===8684.0
9655.0===9655.0
9347.0=

In [ ]:
%%cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include<time.h>
#include <stdio.h>
#include <sys/time.h>
using namespace std;
void MM(float *,float *,float *,int,int,int,int,int,int);
                                      //Kindly input dimensions below in hA,wA,hB,wB,N as described in the assignment to get the desired results and CPU/GPU run-time
 const int hA=500; //row of A
 const   int wA=500;//col of A
 const int hB=500;//row of B
 const   int wB=400;//col of B
 const   int N=100;  // N
 struct timeval t1,t2;
float* A=new float[hA*wA];
float* B=new float[hB*wB];
  float* C=new float[hA*wB];
  float* result=new float[hA*wB];
void initA(int m2, int n2){
    int i2=0;
        for(i2=0 ;i2<(m2*n2);i2++){      
        A[i2]=(rand()%10);    
    }
}
void initResC(int m1 ,int k1){
    int c1=0;
    for(c1=0;c1<(m1*k1);c1++){
        result[c1]=0.0;
        C[c1]=0.0;
    }
}

__global__ void MMGpuKernel(float *Ad,float *Bd,float *Cd,const int wA1,const int wB1){
    float Cval=0.0;
    int r=blockIdx.y*blockDim.y+threadIdx.y;
    int c=blockIdx.x*blockDim.x+threadIdx.x;
    for(int z=0;z<wA1;z++){
        float Adele=Ad[r*wA1+z];
        float Bdele=Bd[z*wB1+c];
        Cval +=(float)(Adele*Bdele);
    }
    Cd[r*wB1+c]=Cval;
}
void MMGpu(float *A,float *B,float *C,int num,int count,const int m,const int n,const int j,const int k){
        gettimeofday(&t1,0);
    float *Ad=new float[m*n];
    float *Bd=new float[j*k];
    float *Cd=new float[m*k];
    cudaMalloc(&Ad,sizeof(int)*m*n);
    cudaMemcpy(Ad,A,sizeof(int)*m*n,cudaMemcpyHostToDevice);
    cudaMalloc(&Bd,sizeof(int)*j*k);
    cudaMemcpy(Bd,B,sizeof(int)*j*k,cudaMemcpyHostToDevice);
    cudaMalloc(&Cd,sizeof(int)*m*k);
    cudaMemcpy(Cd,C,sizeof(int)*m*k,cudaMemcpyHostToDevice);
   // dim3 blocksPerGrid(1,1,1);
   dim3 dimBlock(2,1,1);
   dim3 dimGrid;
    dimGrid.x = ((k + dimBlock.x - 1)/dimBlock.x);
    dimGrid.y = ((m + dimBlock.y - 1)/dimBlock.y);
    
    //gettimeofday(&t1,0);
    MMGpuKernel<<<dimGrid,dimBlock>>>(Ad,Bd,Cd,n,k);
    gettimeofday(&t2,0);
    cudaMemcpy(C,Cd,sizeof(int)*m*k,cudaMemcpyDeviceToHost);
     double executed=(1000000.0*(t2.tv_sec-t1.tv_sec)+t2.tv_usec-t1.tv_usec)/1000.0;
    printf("GPU Execution time for N=%d: %3.4f ms (C%d=%0.1f)\n",count+1,executed,count+1,C[count]);
    //printf("A%d*B=%0.2f \n",(count+1),C[count]);
    cudaFree(Ad);
    cudaFree(Bd);
    cudaFree(Cd);
}
void execCPU(float* result,float* A,float* B,int hA2,int wA2,int wB2,int count2){
        gettimeofday(&t1,0);
        int i1,j1,k1;
        double a1,b1;
 for( i1=0;i1<hA2;i1++){
     for(j1=0;j1< wB2;j1++){
         int sum=0;
         for( k1=0;k1<wA2;k1++){
             a1 =A[i1*wA2+k1];
             b1=B[k1*wB2+j1];
             sum+= a1*b1;
         }
         result[i1*wB2+j1]=sum;
     }
 }
       gettimeofday(&t2,0);

   double executed1=(1000000.0*(t2.tv_sec-t1.tv_sec)+t2.tv_usec-t1.tv_usec)/1000.0;
   printf("CPU Execution time for N=%d: %3.4f ms (C%d=%0.1f)\n",count2+1,executed1,count2+1,result[count2]);
}
int main(void){
    int x1=0;
    int a1=0;
    int count1=0;
                        srand(time(NULL));
    for( x1=0;x1<(hB*wB);x1++){
        B[x1]=(rand()%10);
        
    }
    for(a1=0;a1<N;a1++,count1++){
            initA(hA,wA);
            initResC(hA,wB);
        execCPU(result,A,B,hA,wA,wB,count1);
       MMGpu(A,B,C,N,count1,hA,wA,hB,wB);
        
    }
    
    return 0;
}

CPU Execution time for N=1: 718.8010 ms (C1=10356.0)
GPU Execution time for N=1: 101.1000 ms (C1=10356.0)
CPU Execution time for N=2: 729.1470 ms (C2=10153.0)
GPU Execution time for N=2: 1.0850 ms (C2=10153.0)
CPU Execution time for N=3: 726.9330 ms (C3=10751.0)
GPU Execution time for N=3: 1.1090 ms (C3=10751.0)
CPU Execution time for N=4: 753.3150 ms (C4=9917.0)
GPU Execution time for N=4: 1.1140 ms (C4=9917.0)
CPU Execution time for N=5: 731.4070 ms (C5=10364.0)
GPU Execution time for N=5: 1.1040 ms (C5=10364.0)
CPU Execution time for N=6: 742.4990 ms (C6=10457.0)
GPU Execution time for N=6: 1.0840 ms (C6=10457.0)
CPU Execution time for N=7: 738.2140 ms (C7=9733.0)
GPU Execution time for N=7: 1.1680 ms (C7=9733.0)
CPU Execution time for N=8: 740.3600 ms (C8=9972.0)
GPU Execution time for N=8: 1.1700 ms (C8=9972.0)
CPU Execution time for N=9: 736.4060 ms (C9=10743.0)
GPU Execution time for N=9: 1.1950 ms (C9=10743.0)
CPU Execution time for N=10: 738.4240 ms (C10=9799.0)
GPU Execution 